## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [92]:
import pandas as pd

In [95]:
import time
from datetime import date
from datetime import datetime

In [105]:
data = pd.read_csv('ml-latest-small//ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [113]:
data['date'] = data.timestamp.apply(lambda x: datetime.fromtimestamp(x))
data.head()

,userId,movieId,rating,timestamp,date
0,1,31,2.5,1260759144,2009-12-14 05:52:24
1,1,1029,3.0,1260759179,2009-12-14 05:52:59
2,1,1061,3.0,1260759182,2009-12-14 05:53:02
3,1,1129,2.0,1260759185,2009-12-14 05:53:05
4,1,1172,4.0,1260759205,2009-12-14 05:53:25


In [128]:
users = data.groupby('userId').agg(['count', 'max', 'min'])['date'].reset_index()
users = users.rename(columns={'count':'rates', 'max':'date_max', 'min':'date_min'})
users.loc[users.rates > 100, 'lifetime'] = users.date_max - users.date_min
users[users.rates > 100][['userId', 'lifetime']]

,userId,lifetime
3,4,2 days 08:32:40
7,8,0 days 23:39:47
14,15,5455 days 22:38:16
16,17,0 days 02:14:13
18,19,0 days 01:28:02
...,...,...
655,656,0 days 00:50:53
658,659,365 days 20:23:31
663,664,1136 days 08:20:39
664,665,626 days 12:31:11


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [129]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [130]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [132]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [133]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [149]:
revenue = rzd.merge(auto, how='outer').merge(air, how='outer').fillna(0)
revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [150]:
revenue.merge(client_base, on='client_id', how='left')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Я бы попробовал связать координаты с фактом визита или покупки через дату и время получения координат.
Наиболее близкие значения к дате и времени визита или совершения покупки будут идентификатором для привязки координат. Соответственно, если получится получить эти данные для каждого набора координат в таблице, то можно добавить координаты в таблицу визитов и покупок через связку user_id и даты и времени. 
В случае расчета через время удаленность координат не повлияет на расчеты. Однако, возможно стоит подумать над тем как учитывать географию пользователей в случае, если по координатам визита пользовател находится, к примеру, в Москве, а покупку сделал во Владивостоке.
Из координат можно получить географию визитов и покупок, начиная от стран и заканчивая улицами/домами в одном городе.